In [1]:
pip install TextBlob

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install syllapy 

Note: you may need to restart the kernel to use updated packages.


In [3]:
#import Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import re
from collections import Counter
import syllapy

In [4]:
#Load The DataSet
input_data = pd.read_excel("Input.xlsx")
input_data

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [5]:
print(input_data.dtypes)#shows the Datatypes of the columns 

URL_ID    object
URL       object
dtype: object


In [6]:
print(input_data.shape) #show the number of rows anf columns

(100, 2)


In [7]:
# Function to extract text from a given URL
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article_title = soup.find('h1').get_text().strip()
    article_text = ' '.join([p.get_text() for p in soup.find_all('p')])
    return article_title, article_text

In [8]:
# Function to compute text analysis variables
def compute_text_analysis(article_text):
    # Tokenize the text
    words = re.findall(r'\b\w+\b', article_text.lower())
    sentences = re.split(r'[.!?]', article_text)
    
    # Compute word count
    word_count = len(words)
    
    # Compute sentence count
    sentence_count = len(sentences) - 1
    
    # Compute average number of words per sentence
    avg_words_per_sentence = word_count / sentence_count if sentence_count > 0 else 0
    
    # Compute average sentence length
    avg_sentence_length = sum(len(sentence.split()) for sentence in sentences) / sentence_count if sentence_count > 0 else 0
    
    # Compute complex word count
    complex_words = [word for word in words if syllapy.count(word) > 2]
    complex_word_count = len(complex_words)
    
    # Compute percentage of complex words
    percentage_complex_words = (complex_word_count / word_count) * 100 if word_count > 0 else 0
    
    # Compute fog index
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    
    # Compute syllables per word
    syllables_per_word = sum(syllapy.count(word) for word in words) / word_count if word_count > 0 else 0
    
    # Compute personal pronouns count
    personal_pronouns = Counter(re.findall(r'\b(?:i|me|my|mine|myself|we|us|our|ours|ourselves)\b', article_text.lower()))
    
    # Compute average word length
    avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0
    
    # Compute sentiment analysis scores
    blob = TextBlob(article_text)
    polarity_score = blob.sentiment.polarity
    subjectivity_score = blob.sentiment.subjectivity
    positive_score = len([sent for sent in blob.sentences if sent.sentiment.polarity > 0])
    negative_score = len([sent for sent in blob.sentences if sent.sentiment.polarity < 0])
    
    return positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, \
           percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count, \
           word_count, syllables_per_word, sum(personal_pronouns.values()), avg_word_length

In [9]:
# Will Use the Empty List To Add The Values 
positive_scores = [] 
negative_scores = [] 
polarity_scores = [] 
subjectivity_scores = [] 
avg_sentence_lengths = []
percentage_complex_words = []
fog_indices = []
avg_words_per_sentences = []
complex_word_counts = []
word_counts = []
syllables_per_words = []
personal_pronouns_counts = []
avg_word_lengths = []

In [10]:
#Will Use the below code to trow error
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article_title = soup.title.text.strip() if soup.title else "Title not found"
        article_text = ' '.join([p.text.strip() for p in soup.find_all('p')])
        return article_title, article_text
    except Exception as e:
        print(f"Error occurred while extracting text from {url}: {e}")
        return None, None

In [23]:
# It Will Extract text from URLs and perform text analysis
output_data = []
for index, row in input_data.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, article_text = extract_text_from_url(url)
    if article_text:
        analysis_result = compute_text_analysis(article_text)
        output_data.append([url_id, title, *analysis_result])
    else:
        print(f"Failed to extract text from {url}")

In [28]:
output_data #Will Print The Out_data

[['blackassign0001',
  'Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040. - Blackcoffer Insights',
  21,
  1,
  0.22169253510716927,
  0.5235376412205681,
  18.266666666666666,
  20.46678635547576,
  15.493381208856972,
  18.566666666666666,
  114,
  557,
  1.7576301615798924,
  7,
  5.114901256732495],
 ['blackassign0002',
  'Rising IT Cities and Their Impact on the Economy, Environment, Infrastructure, and City Life in Future - Blackcoffer Insights',
  50,
  13,
  0.12534813530155142,
  0.4273504342448441,
  19.142857142857142,
  24.559270516717323,
  17.480851063829785,
  19.583333333333332,
  404,
  1645,
  1.8595744680851063,
  12,
  5.454103343465046],
 ['blackassign0003',
  "Internet Demand's Evolution, Communication Impact, and 2035's Alternative Pathways - Blackcoffer Insights",
  27,
  9,
  0.108982683982684,
  0.42495861981156086,
  20.737704918032787,
  31.558641975308642,
  20.918538757336574,
  21.24590163934426,


In [29]:
# Will Create A New DataFrame for output data
output_df = pd.DataFrame(output_data, columns=['URL_ID', 'Title', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])

In [33]:
output_df #Will Load the New DataFrame

,URL_ID,Title,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,Rising IT cities and its impact on the economy...,21,1,0.221693,0.523538,18.266667,20.466786,15.493381,18.566667,114,557,1.757630,7,5.114901
1,blackassign0002,Rising IT Cities and Their Impact on the Econo...,50,13,0.125348,0.427350,19.142857,24.559271,17.480851,19.583333,404,1645,1.859574,12,5.454103
2,blackassign0003,"Internet Demand's Evolution, Communication Imp...",27,9,0.108983,0.424959,20.737705,31.558642,20.918539,21.245902,409,1296,2.070216,22,5.997685
3,blackassign0004,Rise of Cybercrime and its Effect in upcoming ...,24,13,0.053151,0.406118,22.339286,29.905808,20.898038,22.750000,381,1274,1.999215,9,5.923077
4,blackassign0005,OTT platform and its impact on the entertainme...,16,4,0.130028,0.493473,20.068182,24.285714,17.741558,20.681818,221,910,1.837363,8,5.541758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,Due to the COVID-19 the repercussion of the en...,30,13,0.067018,0.390397,23.607143,23.173432,18.712230,24.196429,314,1355,1.780074,7,5.284133
96,blackassign0097,Impact of COVID-19 pandemic on office space an...,21,12,0.077763,0.433494,29.720930,17.588326,18.923702,30.279070,229,1302,1.625960,16,4.859447
97,blackassign0098,Contribution of handicrafts (Visual Arts & Lit...,4,1,0.101429,0.377095,38.555556,28.651685,26.882896,39.555556,102,356,1.949438,2,5.755618
98,blackassign0099,How COVID-19 is impacting payment preferences?...,15,4,0.103338,0.398081,21.342857,18.386243,15.891640,21.600000,139,756,1.693122,7,5.087302


In [39]:
# Save output to Output.xlsx
output_df.to_excel('Output_Data_Structure2.xlsx', index=False)

In [36]:
output_df.shape

(100, 15)

In [37]:
print("Successfully Saved The File Output_df")

Successfully Saved The File Output_df
